In [ ]:
from OmniGen import OmniGenPipeline

pipe = OmniGenPipeline.from_pretrained(
    "Shitao/OmniGen-v1",
    quantization_config="bnb_8bit",
    low_cpu_mem_usage=True,
)

In [ ]:
import os
import numpy as np
from PIL import Image

def multi_scale_gen(guidance_scales, img_guidance_scales, outdir, **kwargs):
    os.makedirs(outdir, exist_ok=True)
    all_results = []
    for r, guidance_scale in enumerate(guidance_scales):
        all_results.append([])
        for c, img_guidance_scale in enumerate(img_guidance_scales):
            result = pipe(
                guidance_scale=guidance_scale,
                img_guidance_scale=img_guidance_scale,
                **kwargs,
            )[0]
            result.save(f"{outdir}/{guidance_scale:.1f}_{img_guidance_scale:.1f}.png")
            all_results[r].append(np.array(result))
        all_results[r] = np.concatenate(all_results[r], axis=1)
    all_results = np.concatenate(all_results, axis=0)
    all_results = Image.fromarray(all_results)
    all_results.save(f"{outdir}/{outdir}.png")
    return all_results

In [ ]:
prompt = "ducks"
result_0 = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    num_inference_steps=50,
    guidance_scale=3,
    seed=0,
)[0]
result_0.save("result_0.png")
result_0

In [ ]:
prompt = "<img><|image_1|><img>\n Change the color of the duck to black."
size = 512
step = 20
multi_scale_gen(
    outdir="outputs",
    prompt=prompt,
    input_images=["result_0.png"],
    height=size,
    width=size,
    max_input_image_size=size,
    num_inference_steps=step,
    guidance_scales=np.linspace(2.0, 3.0, 3),
    img_guidance_scales=np.linspace(1.0, 2.0, 3),
    seed=1,
)